In [513]:
import pandas as pd
import archook

# Use archook to import arcpy
archook.get_arcpy()
import arcpy
from arcpy.sa import *
# calculate upslope on an Emme network

# Automating Slope Calcs

- export network shapefile from emme using API
    - make sure we can set a coordinate system here (projected state plane washington)

### Input: emme network shapefile

In [527]:
# load Emme network shapefile
in_fc = r'R:\Bike\slope\bkr\bkr_network.gdb\links'
# in_fc = r'R:\Bike\slope\bkr\bkr_sample.gdb\bkr_sample_links'

In [528]:
# Find two-way links - we only need to split these links once
emme_links = arcpy.da.FeatureClassToNumPyArray(in_fc, ('ID','LENGTH','MODES','INODE','JNODE'))
df = pd.DataFrame(emme_links)

In [ ]:
# loop through each link 
ij_links = []
ji_links = []

for rownum in xrange(len(df)):
    inode = df.iloc[rownum].INODE
    jnode = df.iloc[rownum].JNODE
    ij_df = df[(df['INODE']==inode)&(df['JNODE']==jnode)]
    if len(ij_df) == 1:
        ij_id = ij_df.ID.values[0]
        
    
#     ij_id = df[(df['INODE']==inode)&(df['JNODE']==jnode)].ID.values[0]
    ji_df = df[(df['INODE']==jnode)&(df['JNODE']==inode)]
    if len(ji_df) == 1:
        ji_id = ji_df.ID.values[0]
    
    if ji_id not in ij_links:
        ij_links.append(ij_id)

1 6091
1 6564
2 1820
2 6096
3 1822
4 6094
4 6100
5 6099
5 6105
5 6107
5 6902
6 6103
6 6107
6 6109
6 6115
7 6104
7 6109
7 6114
8 6111
8 6113
9 6112
9 6113
9 6118
10 6110
10 6112
10 6114
10 6119
11 5987
11 6124
11 6129
12 6125
12 6153
12 6164
13 6147
14 6154
14 6165
15 6166
16 7129
17 6156
17 7129
18 6148
18 7131
19 2435
20 6150
20 6168
20 7808
21 6152
21 6168
22 6145
22 6152
22 6159
22 6162
23 6142
23 6146
23 6160
23 6162
24 6142
24 6172
25 1819
25 6171
26 6150
26 7131
26 7807
27 1819
27 7132
28 4997
28 7132
29 4997
29 6151
30 6137
30 6144
30 6149
30 6151
31 6149
31 6167
31 7807
32 6147
32 6167
33 5906
33 6129
33 6131
33 6136
34 1738
34 6131
34 6133
34 6137
35 6133
35 6342
36 6342
36 9050
37 6169
37 9050
38 5734
38 6134
38 6135
39 6122
39 6132
39 6134
40 1738
40 6121
40 6130
40 6132
41 5915
41 6130
42 6108
42 6110
42 6115
42 6120
43 6121
43 6367
43 6368
43 6902
44 6095
44 6106
44 6122
44 6903
45 6092
45 6095
45 6097
45 6102
46 6092
47 6178
48 5545
49 5546
49 5547
50 5794
50 6176
51 5548

In [536]:
df[df['ID'] == '6564-1']

,ID,LENGTH,MODES,INODE,JNODE


In [520]:
ji_df = df[df['ID'].isin(ij_links)]
ij_df = df[-df['ID'].isin(ij_links)]

In [ ]:
df

In [388]:
ij_df.head()

,ID,LENGTH,MODES,INODE,JNODE
60,5545-48,0.032842,lsuvtbjpmgheidxa,5545,48
61,5545-5189,0.065014,lsuvtbjpmgheidwa,5545,5189
63,5546-49,0.039479,lsuvtbjpmgheidxa,5546,49
64,5546-52,0.085170,lsuvtbjpmgheidxa,5546,52
67,5547-49,0.041890,lsuvtbjpmgheidxa,5547,49


In [389]:
ji_df.head()

,ID,LENGTH,MODES,INODE,JNODE
0,48-5545,0.032842,lsuvtbjpmgheidxa,48,5545
1,49-5546,0.039479,lsuvtbjpmgheidxa,49,5546
2,49-5547,0.041890,lsuvtbjpmgheidxa,49,5547
3,50-5794,0.042924,lsuvtbjpmgheidxa,50,5794
4,50-6176,0.038451,lsuvtbjpmgheidxa,50,6176


In [ ]:
# Loop over ij_df only for creating points,
# later we will want to loop over the entire dataframe results

In [294]:
# export the ij_df as a new feature class 
arcpy.env.workspace = r'R:\Bike\slope\bkr\bkr_sample.gdb'
point_out_new = r'ij_links'
arcpy.CopyFeatures_management(ij_df, point_out_new) 

<Result 'R:\\Bike\\slope\\bkr\\bkr_sample.gdb\\ij_links'>

- split network lines into points
- note: this takes 30-60 minutes

- should only have to do this one time
- subsequent buffering will compare the imported shapefile links to the result of the cold-start process and only work on the links that haven't been processed yet, appending them to the existing results

In [300]:
# Loop through each network link and split the line into points, saving them in points array
points = []
sr = arcpy.Describe(in_fc).spatialReference
counter = 0

# Set the step based on the line's length
# How many times must the line be split
# 
# Using 30 of fidelity - data is available every 30 meters so this may be unnecessary
segment_len = 30

# Create seperate array that holds tuple of link ID and point coordinates (in state plane coords)
final_result = []
count = 0
with arcpy.da.SearchCursor(in_fc,["SHAPE@",'ID'], spatial_reference=sr) as cursor:  
    for row in cursor:
        # Only process IJ links, because JI are exactly the same polyline shape
        if row[1] in ij_links:
            count += 1
            split_count = int(row[0].length/segment_len)
            big_output = []
            for i in range(split_count):
                point = row[0].positionAlongLine(i*segment_len)
                points.append(point)
                x = point.firstPoint.X
                y = point.firstPoint.Y
                point_list = []
                point_list.append(x)
                point_list.append(y)
                output = (str(row[1]), (x, y))
                final_result.append(output)
                #local_points.append(points)
            #my_dict[row[1]] = local_points

- Export results to a feature class called detailed_points

In [303]:
point_out_new = r'\link_components'
arcpy.CopyFeatures_management(points, r"R:\Bike\slope\bkr\bkr_sample.gdb" + point_out_new) 

<Result 'R:\\Bike\\slope\\bkr\\bkr_sample.gdb\\link_components'>

- Intersect the points with the links to get the edge IDs

In [304]:
arcpy.env.workspace = r'R:\Bike\slope\bkr\bkr_sample.gdb'
inFeatures = ["link_components", "bkr_sample_links"]
intersectOutput = "link_components_full"
clusterTolerance = 1.5    
arcpy.Intersect_analysis(inFeatures, intersectOutput, "", clusterTolerance, "point")

<Result 'R:\\Bike\\slope\\bkr\\bkr_sample.gdb\\link_components_full'>

- Intersect with a raster to get elevation
- import elevation raster from W:/geodata/raster/dem30m
    - this is the raster of elevations at 30 m fidelity
    - note that elevation values are in METERS

In [305]:
# Note that spatial analyst must be active for this portion
arcpy.CheckOutExtension("Spatial")
# arcpy.env.workspace = r'R:\Bike\slope\bkr\bkr_network.gdb'

in_point_features = r'link_components_full'
in_raster = r'W:\geodata\raster\dem30m'
out_point_features = r'link_components_elevation'
ExtractValuesToPoints(in_point_features, in_raster, out_point_features)

<geoprocessing server result object at 0x1a7607d0>

In [391]:
# Read resulting intersection of points with elevation into numpy/pandas
elevation_shp = arcpy.da.FeatureClassToNumPyArray(out_point_features, ('RASTERVALU','ID','LENGTH','MODES','INODE','JNODE'))
df = pd.DataFrame(elevation_shp)

In [392]:
# List of links IDs
link_list = df.groupby('ID').min().index

In [393]:
# Loop through all edges
# Assume that all links are bi-directional and compute ij and ji direction slopes
# if a line is truly one-way, we will discard the ji direction
# since most are two-way it's worth it calculate for all links and merge results later
upslope_ij = {}
upslope_ji = {}
for link in link_list: # Replace [1] with edges when ready to loop over everything
    link_df = df[df['ID'] == link]

    # Extract the elevation data to numPy because it's faster to loop over
    elev_data = link_df['RASTERVALU'].values

    # Loop through each point in each edge
    upslope_ij[link] = 0
    upslope_ji[link] = 0
    for point in xrange(len(elev_data)-1):  # stop short of the list because we only want to compare the 2nd to last to last
        elev_diff = elev_data[point+1] - elev_data[point]
        if elev_diff > 0:
            upslope_ij[link] += elev_diff
        elif elev_diff < 0:
            upslope_ji[link] += abs(elev_diff)      # since we know it will be "negative" for the JI direction when calculated
                                                    # in references to the IJ direction

In [509]:
# Import dictionary to a series and attach upslope back on the original dataframe
upslope_ij_s = pd.Series(upslope_ij, name='elev_gain_ij')
upslope_ji_s = pd.Series(upslope_ji, name='elev_gain_ji')
upslope_ij_s.index.name='ID'
upslope_ij_s = upslope_ij_s.reset_index()
upslope_ji_s.index.name='ID'
upslope_ji_s = upslope_ji_s.reset_index()

# Attach ij-direction slope to IJ links
slope_ij = pd.merge(ij_df,upslope_ij_s,on='ID')
slope_ij.rename(columns={"elev_gain_ij": "elev_gain"}, inplace=True)

# Attach ji-direction slope to JI links

# fo JI links, flip the i and j values to get lookup of ji links
upslope_ji_s['newID'] = upslope_ji_s.ID.apply(lambda row: row.split('-')[-1]+"-"+row.split('-')[0])
slope_ji = pd.merge(ji_df,upslope_ji_s,left_on='ID',right_on='newID')
slope_ji.rename(columns={"elev_gain_ji": "elev_gain"}, inplace=True)
slope_ji['ID'] = slope_ji['newID']
slope_ji.drop(['ID_x','ID_y','newID'],axis=1,inplace=True)

# Append ji rows to ij to get a complete list of links
slope_df = slope_ij.append(slope_ji)

# Convert elevation into feet from meters
slope_df['elev_gain'] = slope_df['elev_gain']*3.28084

In [510]:
# Calcualte the average upslope in feet/feet
# Network distance measured in: miles, elevation in meters 
slope_df['avg_upslope'] = slope_df['elev_gain']/(slope_df['LENGTH']*5280)

- reformat and export as emme_attr.in
- for BKR, assume all bike facilities are 0 for now

In [511]:
emme_attr = slope_df
emme_attr.rename(columns={'INODE':'inode','JNODE':'jnode','avg_upslope':'@upslp'},
                inplace=True)
# Export as csv to join to ArcMap
emme_attr.to_csv('upslope.csv',index=False)

In [501]:
emme_attr.drop(['ID','LENGTH','MODES','elev_gain'], axis=1, inplace=True)

ValueError: labels ['ID' 'LENGTH' 'MODES' 'elev_gain'] not contained in axis

In [494]:
# add bike facility of 0
emme_attr['@bkfac'] = 0

In [499]:
emme_attr.to_csv('emme_attr.in', sep=' ', index=False)



In [496]:
# Load the soundcast bkr comparison
df = pd.read_csv(r'R:\SoundCast\Inputs\2014\bikes\emme_attr.in', sep=' ')

In [497]:
print df['@upslp'].max()
print df['@upslp'].mean()
print df['@upslp'].median()

22.3756189894
0.0332534048304
0.0192160023467


In [498]:
print emme_attr['@upslp'].max()
print emme_attr['@upslp'].mean()
print emme_attr['@upslp'].median()

0.213796218503
0.0247169109468
0.00969354833827
